In [1]:
import gc
import os
import nltk
import tqdm
import numpy as np
import pandas as pd
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Eshanka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.engine import Layer
from keras.layers import Activation, Add, Bidirectional, Conv1D, Dense, Dropout, Embedding, Flatten
from keras.layers import concatenate, GRU, Input, LSTM, MaxPooling1D
from keras.backend import backend as K
from keras.layers import GlobalAveragePooling1D,  GlobalMaxPooling1D, SpatialDropout1D
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing import text, sequence
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss
from sklearn.model_selection import train_test_split
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks

In [3]:
gru_len = 128
Routings = 5
Num_capsule = 10
Dim_capsule = 16
dropout_p = 0.3
rate_drop_dense = 0.3

In [4]:
def squash(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return x / scale

In [95]:
df = pd.read_csv('Datasets/dga_data.csv', sep = ',')
df = pd.DataFrame(df)
print(df)

        isDGA                      domain                            host  \
0         dga  6xzxsw3sokvg1tc752y1a6p0af  6xzxsw3sokvg1tc752y1a6p0af.com   
1         dga              glbtlxwwhbnpxs               glbtlxwwhbnpxs.ru   
2         dga              xxmamopyipbfpk               xxmamopyipbfpk.ru   
3         dga   zfd5szpi18i85wj9uy13l69rg   zfd5szpi18i85wj9uy13l69rg.net   
4         dga                jpqftymiuver                 jpqftymiuver.ru   
...       ...                         ...                             ...   
159995  legit                     manhunt                     manhunt.net   
159996  legit                      yupptv                      yupptv.com   
159997  legit                    bbcmundo                    bbcmundo.com   
159998  legit                   kanogames                   kanogames.com   
159999  legit                   roadrover                    roadrover.cn   

            subclass  
0        gameoverdga  
1       cryptolocker  
2     